# <center>Preprocessing Data Analysis</center>

In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import csv
import os
from tqdm.auto import tqdm
import gc

print("Importing spike interface, this may take a while...")
import spikeinterface.full as si
print("Done...")
import scipy

import probeinterface as pi
from probeinterface.plotting import plot_probe_group, plot_probe

# Load probe
probe = pi.read_prb('ressources/mea_256_30-8iR-ITO.prb')

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('qt5agg')

from utils import *         # Local file containing all the functions that we need
import params               # Parameters file. You should tune it for your own experiment

Importing spike interface, this may take a while...
Done...


# Run this before the sorting

### Cell 1: Create Symbolic links

As spyking circus requires a specific file format, we create here symbolic links to the raw recording file with the right Spyking circus format. This cell contains important variable for the rest of the notebook. You should check stim duration as sanity check.

In [3]:
"""
    Inputs
"""

#Duration at the begining of the recording to check recording type (default 10s)
time = 40   #Increase until all your holographic recordings are detected as such


    
"""
    Variables
    
    DO NOT CHANGE VALUES HERE UNLESS DEBUG/SPECIFIC USE
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#Path to your sorting directory where symbolic links should be created
symbolic_link_directory = params.symbolic_link_directory

#Link to the actual raw files from the recording listed in the input_file
recording_directory     = params.recording_directory

#Loading raw recording files names
recording_names = params.recording_names

#Type of data in your raws (float32, uint16, int16, ...)
dtype = params.dtype

#Sampling frequency of the mea
fs = params.fs



"""
    Processing
"""

#Paths to which create recording links (useful if raws needs to be filtered before sorting)
if os.path.isdir(params.raw_filtered_directory) and input("Filtered raws have been found. Use these for symbolic links ? type Yes :") in ["YES", "yes", "Yes", "Y", "y"]:
    link_recording_directory     = params.raw_filtered_directory
    link_recording_names         = params.find_files(link_recording_directory) 
else :
    link_recording_directory     = recording_directory
    link_recording_names         = recording_names
    
    
print(f"Links created from files in : {link_recording_directory}\n")
link_names, previously_existing = create_symlinks([rec+r'.raw' for rec in link_recording_names], symbolic_link_directory, link_recording_directory, print_warning=False)

recording_names = [rec.replace('.raw','') for rec in recording_names]
rec_it = recording_names[:]+['end']
print('Number of recordings: {}\n'.format(len(recording_names)))

#getting onset for next prints
onsets = {}
onsets = recording_onsets(recording_names, path = recording_directory)


#Opening files
print('\nCheck that recordings lengths are consistent with recording names\n') 

for i in range(len(rec_it)-1):
    is_holo = ["HOLO RECORDING" if is_holographic_rec(os.path.join(recording_directory,recording_names[i]+r".raw"), dtype=dtype, probe_size = time*params.fs) else "Visual Recording"]
    print("{} : \t{} \t---> link {}".format(i,rec_it[i], [previously_existing[i] if previously_existing[i] == ' already existed' else "created"][0]))
    print("\t{} --> {} minutes \n".format(is_holo[0],
                                                         int((onsets[rec_it[i+1]]-onsets[rec_it[i]])/fs/60)))      

"""Output :

Var :
recordings_names : Ordered list of stimuli names played during experiment and their detected types
"""   

print('\n\t\t\t------ End Of Cell ------')

Filtered raws have been found. Use these for symbolic links ? type Yes :Y
Links created from files in : /home/guiglaz/Documents/Pipeline_Git_Repo/RAW_filtered

Number of recordings: 22


Check that recordings lengths are consistent with recording names

0 : 	00_Checkerboard_30ND50%_20pix30checks_30Hz 	---> link  already existed
	Visual Recording --> 55 minutes 

1 : 	01_Checkerboard_30ND50%_16pix40checks_30Hz 	---> link  already existed
	Visual Recording --> 24 minutes 

2 : 	02_DG_30ND50%_2sT_50Hz 	---> link  already existed
	Visual Recording --> 6 minutes 

3 : 	03_Chirp_20reps_30ND50%_50Hz 	---> link  already existed
	Visual Recording --> 11 minutes 

4 : 	04_Flicker_BeforeDrugs_30ND50%_1Hz 	---> link  already existed
	Visual Recording --> 5 minutes 

5 : 	05_VDH_Synchro+MultiSpots(bright)_N8_Z(-35)_30ND50%_40Hz 	---> link  already existed
	HOLO RECORDING --> 115 minutes 

6 : 	06_VDH_Synchro_N10_Z(-35)_30ND50%_40Hz 	---> link  already existed
	HOLO RECORDING --> 16 minutes 

7 : 	0

### Cell 2 : Sanity checks

#### <center>REQUIRES CELL 1 RUN</center>

Sanity checks of symlinks comparing onsets recorded from raw files and onsets recorded from symlinks. If both plots are identical, it is likely that links point to the right recording files. Otherwise, links are pointing to the wrong files. You may want to check your params.py file for a wrong directory.

In [4]:
"""
    Variables
    
    DO NOT CHANGE VALUES HERE UNLESS DEBUG/SPECIFIC USE
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#Link to the folder where spiking circus will look for the symbolic links "recording_0i.raw"
symbolic_link_directory = params.symbolic_link_directory

#Link to the actual raw files from the recording listed in the input_file
recording_directory     = params.recording_directory


"""
    Processing
"""

rec_onsets  = recording_onsets(recording_names, path = recording_directory)

link_onsets = recording_onsets(link_names, path = symbolic_link_directory)

"""
    Ploting
"""
print("\nMake sure both plots are identical")
plt.close('all')
fig = plt.figure("Links Sanity Check", figsize=(10,6))

# LEFT PLOT
ticks=[]
plt.subplot(1,2,1)
for rec in recording_names:
    plt.axhline(rec_onsets[rec]/params.fs/60)
    ticks.append(rec_onsets[rec]/params.fs/60)
plt.axhline(rec_onsets['end']/params.fs/60)
ticks.append(rec_onsets['end']/params.fs/60)

plt.yticks(ticks)
plt.title('Recordings onsets')
plt.ylabel('Minutes')

# RIGHT PLOT
ticks=[]
plt.subplot(1,2,2)
for rec in link_names:
    plt.axhline(link_onsets[rec]/params.fs/60)
    ticks.append(link_onsets[rec]/params.fs/60)
plt.axhline(link_onsets['end']/params.fs/60)
ticks.append(link_onsets['end']/params.fs/60)
plt.yticks(ticks)
plt.title('Links onsets')
plt.ylabel('Minutes')

plt.show(block = False)

print('\n\t\t\t---symlinks--- End Of Cell ------')


Make sure both plots are identical

			---symlinks--- End Of Cell ------


### Cell 3 : Extract the triggers

#### <center>REQUIRES CELL 1 RUN</center>

Extract triggers from either the visual or holo trigger channel. Automatic detection of Holographic recording. Check that the detection is perform on the right files. Perform triggers sanity checks for visual stimumi. You can plot them later on cell 4. Can take up to more than 1h to run all recordings depending on your experiment length.

In [6]:
"""
    Inputs
"""

#you can decide here to extract the triggers only for some recordings. List their indexes here (starting from 0).
select_rec = []    # do only measurement N, put [] or the complet list to call all of them

#You can manually enter here the recording numbers for which you want to manually select the recording type. 
rec_type_manual_selection = []

#Erase already extracted triggers. If False, you'll have to manually confirm that you want to erase triggers 
extract_again = False

"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you wan
    t to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#name of your experiment for saving the triggers
exp = params.exp                     

#the optimal threshhold for detecting stimuli onsets varies with the rig
threshold = params.threshold

# number of triggers samples acquired per second
fs         = params.fs

#The folder in which you want your triggers to be saved 
triggers_directory = params.triggers_directory

#Channel recording triggers in case of holographic stimuli
holo_channel_id   = params.holo_channel_id

#Channel recording triggers in case of visual stimuli
visual_channel_id = params.visual_channel_id 

#Datatype of the recording. If raws from the mcd files, put "uint16". If filtered raws, put "int16"
dtype = params.dtype


"""
    Processing
"""

for rec in range(len(recording_names)):

    if select_rec:
        if rec not in select_rec: continue
    
    print('\n-------------   Processing recording {} out of {}   -------------\n'.format(rec+1,len(link_names)))


    # Creating all files path
    input_file    = os.path.join(recording_directory,recording_names[rec]+r".raw")
    trigger_file  = os.path.join(triggers_directory,'{}_{}_triggers.pkl'.format(exp,recording_names[rec]))
    data_file     = os.path.join(triggers_directory,'{}_{}_triggers_data.pkl'.format(exp,recording_names[rec]))
    
    print('The triggers are extracted from the sorting file:\t{}\nand the results will be saved at:\t\t\t{}'.format(recording_names[rec],trigger_file))

    if os.path.exists(data_file) and not extract_again:
        if (str(input('Trigers already extracted previously. Write again files files? Type Yes to do so :\n')) not in ["Y", "y", "yes", "Yes"]) : continue
    
    if rec in rec_type_manual_selection:
        if(str(input("Is this an holographic recording ? If yes, type Y :")) in ["Y", "y", "yes", "Yes"]):
            is_holo = True
        else :
            is_holo = False
    else :
            is_holo = is_holographic_rec(input_file, dtype=dtype, probe_size = time*fs)
    
    if is_holo: 
        #in this case the stimulus was holograpic
        print(r" /!\ HOLOGRAPHIC Recording /!\ ")
        channel_id   = holo_channel_id
        trigger_type = 'holo'
        onsets_file  = os.path.join(triggers_directory,'{}_{}_laser_onsets.pkl'.format(exp,recording_names[rec]))
        offsets_file  = os.path.join(triggers_directory,'{}_{}_laser_offsets.pkl'.format(exp,recording_names[rec]))

    else: 
        #in this other case the stimulus was visual
        print(r" /!\ VISUAL Recording /!\ ")
        channel_id   = visual_channel_id        
        trigger_type = 'visual'
        
    #Processing of data calling utils functions
    print("Loading Data...")
    data, t_tot    = load_data(input_file, channel_id = channel_id, dtype=dtype) 
                                      
    indices        = detect_onsets(data,threshold)
    indices_errors = run_minimal_sanity_check(indices, stim_type = trigger_type)
    
    #Saving data using utils function save_obj
    save_obj({'indices':indices,'duration':t_tot,'trigger_type':trigger_type,'indice_errors':indices_errors}, trigger_file )
    save_obj(data,data_file)

    if trigger_type == 'holo':
        offsets = detect_offsets(data)
        
        save_obj(indices, onsets_file)
        save_obj(offsets, offsets_file)
        

"""
    Output
    
    Saved in triggers_directory :

{experience_name}_{link_file_name}_triggers.pkl (dict) : 
    keys 'indices'  --> detected triggers indices, 
         'duration' --> the stimuli duration, 
         'trigger_type'  --> the detection visual or holo stimuli, 
         'indice_errors' --> triggers violating sanity check 
         
{experience_name}_{link_file_name}_triggers_data.pkl (numpy array) : raw signal recorded on the trigger channel

"""
print('\n\t\t\t------ End Of Cell ------')


-------------   Processing recording 1 out of 22   -------------

The triggers are extracted from the sorting file:	00_Checkerboard_30ND50%_20pix30checks_30Hz
and the results will be saved at:			/home/guiglaz/Documents/Pipeline_Git_Repo/Analysis/triggers/Pipeline_DEV_00_Checkerboard_30ND50%_20pix30checks_30Hz_triggers.pkl
Trigers already extracted previously. Write again files files? Type Yes to do so :
y
 /!\ VISUAL Recording /!\ 
Loading Data...


  0%|          | 0/66018000 [00:00<?, ?it/s]

KeyboardInterrupt: 

### CELL 4 : Plots triggers for sanity check

#### <center>REQUIRES CELL 1 RUN & CELL 3 RUN AT LEAST ONCE FOR THIS EXPERIMENT </center>

Plots the raw trigger signal with the detected triggers and the errors detected. Independently, plots also the detected triggers, should be a perfect diagonal. Third, plots the number of time points gap to the most common trigger duration (ie theoretical_time_per_frame +- ploted value).

#### <center>/!\/!\/!\ Caution on memory leaks /!\/!\/!\ </center>

In [17]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#Experiment name
exp = params.exp

# Optimal threshhold for detecting stimuli onsets varies with the rig
threshold  = params.threshold

# Directory where plots will be saved
output_directory = params.output_directory


"""
    Inputs
"""

#Set True if you want the plots to be saved
save = False

#Define your x-axis ploting window in a tuple (x-min,x-max). Set False to plot the complete data
ploting_range = False


"""
    Ploting
"""

print(*['{} : {}'.format(i,recording_name) for i, recording_name in enumerate(recording_names)], sep="\n")
recordings = [int(rec_id) for rec_id in input("\nSelect recording : ").split()]


plt.close('all')
gc.collect()
plot_idx = 0

for rec in recordings:
    plot_idx+=1
    # Loading data from pickle files created in cell 3
    data    = np.array(load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers_data.pkl'.format(exp,recording_names[rec])))))
    extracted = load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers.pkl'.format(exp,recording_names[rec]))))
    err = extracted['indice_errors']
    indices = extracted['indices']
    rec_type = extracted["trigger_type"]
    
    # If ploting range is a tuple, reduce the plot to indices between both values of the tuple
    if ploting_range :
        indices = indices[np.logical_and(indices > ploting_range[0], indices < ploting_range[1])]
        data    = data[np.logical_and(np.array(range(len(data))) > ploting_range[0], np.array(range(len(data))) < ploting_range[1])]
        err     = err[np.logical_and(err > ploting_range[0], err < ploting_range[1])]
    
    plt.figure("Trigger sanity check {}".format(plot_idx))
    
    # Top plot with raw trigger signal, threshold of detection, detected triggers and wrong triggers
    plt.subplot(2,1,1)
    plt.title('{}\n{} channel'.format(recording_names[rec],rec_type))
    
    plt.plot(np.linspace(0,len(data)/fs,len(data) ),data)
    plt.plot(indices/fs,data[indices],'.',markersize=2,zorder=10)

    plt.axhline(threshold, color='green')
    plt.scatter(err/fs,data[err], color='red', marker='x',zorder = 15)
    
    # Bottom left plot of triggers indices. Shoule be a perfect diagonal
    plt.subplot(2,2,3)
    plt.plot(indices)
    plt.title('Detected indices')
    
    # Bottom right plot of relative error gap between detect time of frame and mean frame time
    plt.subplot(2,2,4)
    plt.plot(np.diff(np.diff(indices)))
    try :
        plt.title('Duration {} +- error'.format(np.round(np.mean(np.diff(indices)))))
    except :
        plt.title('Duration {} +- error'.format("NOT COMPUTED"))
                  
    plt.tight_layout()
    plt.show(block = False)
    
    # Saving plot if needed
    if save:
        fig_name = os.path.join(output_directory,r'{}_{}.png'.format(recording_names[rec],link_names[rec]))
        plt.savefig(fig_name)


"""
    Output
    
    if save == True
    
    {recording_file_name}_{link_file_name}.png : Plots for a given recording file

"""

print('\n\t\t\t------ End Of Cell ------')

0 : 00_AccCechkerboard_30Hz_20px_30sq_50%-30ND
1 : 01_Checkerboard_30Hz_16px_40sq_50%-30ND
2 : 02_Chirp_50Hz_50%-30ND
3 : 03_DG_50Hz_50%-30ND
4 : 04_Flicker_BeforeLAP4+ACET_30ND50%_1Hz
5 : 05_Flicker_AfterLAP4+ACET+30min_30ND50%_1Hz
6 : 06_HoloStim1_LAP4+ACET_Z(-30)
7 : 06_HoloStim1_LAP4+ACET_Z(-30)0001
8 : 07_Flicker_1Hz_50%-15ND_After-LAP4-ACET-Before-SR95531
9 : 08_Flicker_1Hz_50%-15ND_After-LAP4-ACET-SR95531-t30
10 : 09_HoloStim2_LAP4+ACET+SR95531_Z(-30)

Select recording : 10

			------ End Of Cell ------


### Cell 5 : Dead times file creation

#### <center>REQUIRES CELL 1 RUN & CELL 3 RUN AT LEAST ONCE FOR THIS EXPERIMENT </center>

Run minimal sanity check on visual triggers and create the dead times in sec for holographic stimuli. If you only have visual triggers, running this cell isn't mandatory, it only provides a repeated sanity check for visual stims...

In [ ]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

#name of your experiment for saving the triggers
exp = params.exp

#The folder in which you saved your triggers
triggers_directory = params.triggers_directory

#The folder containing raw recording files
recording_directory = params.recording_directory

# Directory where dead_times file will be saved
output_directory = params.output_directory
"""
    Processing
"""
plt.close("all")
gc.collect()

rec_onsets    = recording_onsets(recording_names, path = recording_directory)  

holo_onsets   = []
holo_triggers = []
for rec in range(len(recording_names)):
    
    #Loading data
    trig_data = load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers.pkl'.format(exp,recording_names[rec]))))
    
    
    if trig_data['trigger_type']=='holo':
        #If recording was detected as holographic
        print('Recording {} detected as holographic :\t{}'.format(rec, recording_names[rec]))
        holo_onsets.append(rec_onsets[recording_names[rec]])
        holo_triggers.append(trig_data['indices'])
        print('Number of triggers to be added to dead times file : {}'.format(len(trig_data['indices'])))
        
    else:
        #Otherwise it is considered as visual
        print('Recording {} :\t{}'.format(rec, recording_names[rec]))
        err = run_minimal_sanity_check(trig_data['indices'])    
    print('\n')

print('Writting dead times file for {} holographic recordings...'.format(len(holo_triggers)))
#Writting .dead file using utils function
write_dead_times_file(holo_triggers,holo_onsets, output_directory)


"""
    Ploting
"""

#Potting once more the recordings onsets similarly to cell 2
ticks=[]
plt.ioff()
plt.close('all')
plt.figure() 
for rec in range(len(recording_names)):
    plt.axhline(rec_onsets[recording_names[rec]]/params.fs/60)
    ticks.append(rec_onsets[recording_names[rec]]/params.fs/60)
    
plt.yticks(ticks)
plt.title('Recording starting time')
plt.ylabel('Minutes')
plt.show()


"""
    Output
    
    {recording_file_name}_dead_times.dead : dead time file given times to exclude in spyking circus

"""

print('\n\t\t\t------ End Of Cell ------')

Recording 0 :	1-Checkerboard_40hz_12px_60sq.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 1 :	2-Chirp_50hz.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 2 :	3-DG_50hz.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 3 :	3-DG_50hz_bis.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 4 :	4-Moving_disks_r7_40hz.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 5 :	5-Moving_disks_r10_40hz.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 6 :	6-Moving_disks_r14_40hz.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 7 :	7-Chirp_End_50hz.raw
No sanity check performed, only 1 trigger detected. Is threshold correct ?


Recording 8 :	8-Checkerboard_End_40hz_12px_60sq.raw
No sanity check performed, only 1

## Run Spyking Circus 1 sorting algorithm

Run spyking circus:<br>
<br> >> conda activate circus1'
<br> >> spyking-circus recording_00.raw -p
<i>(if everything looks good continue)</i>
<br> >> spyking-circus recording_00.raw -c 10
<i>(run the sorting on half of your cores)</i>
<br> >> spyking-circus rcording_00.raw -c 10 -m converting <i>(export to phy's format)</i>

# <center>/!\/!\/!\ Run this after the sorting /!\/!\/!\ </center>

### Cell 6 : Creating all clusters rasters plots

#### <center>REQUIRES CELL 1 RUN</center>

Run after automatic sorting to help with manual sorting. Saves all automatic clusters' rasters on the repeated checherboard in the phy directory. You can run this several times during sorting to make new clusters rasters. Can take few sec per cluster...

In [22]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""
#length of each sequence composed of half repeated sequence and half random sequence

nb_frames_by_sequence = params.nb_frames_by_sequence

# Name of your experiment
exp = params.exp

#Path to the folder with the phy output
phy_directory = params.phy_directory

#Path to raw recording files
recording_directory = params.recording_directory

#Frequency of sampling of the mea
fs = params.fs

"""
    Input
"""
#Number of the checkerboard recording of choice (start from zero)
check_recording_number = 0

#Checkerboard frequency in Hz
stimulus_frequency = 30

"""
    Processing
"""
###################################
#### Loading phy clusters info ####
###################################
plt.close("all")
gc.collect()


rec_onsets = recording_onsets(recording_names, path = recording_directory)  

# Get cells index and number
cluster_number , good_clusters = extract_cluster_groups(phy_directory)

# Extract the spike times from the spike sorting files. This can take a few minutes.
print('Spike extraction: ')
all_spike_times = extract_all_spike_times_from_phy(phy_directory)

print('Cluster to Spikes matching: ')
# create a dictionary with another dictionary for each cluster
all_neurons_data = split_spikes_by_recording(all_spike_times, cluster_number, rec_onsets)


#############################
#### Making raster plots ####
#############################

print(*['{} : {}'.format(i,recording_name) for i, recording_name in enumerate(recording_names)], sep="\n")
check_recording_number = int(input("\nSelect the checkerboard recording  for the raster plots: "))

checkerboard_name = recording_names[check_recording_number]

checkerboard_spikes = get_recording_spikes(checkerboard_name, all_neurons_data)

trig_data = load_obj(os.path.normpath(os.path.join(params.triggers_directory,'{}_{}_triggers.pkl'.format(exp,checkerboard_name))))
triggers = trig_data['indices']/fs
raster_data = {}

print('Building Raster plots : ')
for (cell_nb, spike_times) in tqdm(checkerboard_spikes.items()):
    # Align triggers and spike times
    aligned_triggers, aligned_spike_times = align_triggers_spikes(triggers, spike_times)
    
    # Get rasters on repeated sequence
    raster_data[cell_nb] = build_rasters(aligned_spike_times, aligned_triggers, stim_frequency = stimulus_frequency)


"""
    Saving
"""

#Save all clusters rasters plots    
fig_directory = os.path.normpath(os.path.join(phy_directory,r'clusters_rasters_{}'.format(check_recording_number)))
if not os.path.isdir(fig_directory): os.makedirs(fig_directory)

    
print("Saving rasters plots of all clusters :")
for cell_nb in tqdm(raster_data.keys()):
    fig, axs = plt.subplots(nrows = 2,ncols = 1, sharex=True, gridspec_kw={'height_ratios': [3, 1]}, figsize=(10,10))

    plt.suptitle(f'Cell {cell_nb}')

    ax_rast = axs[0]
    ax_rast.eventplot(raster_data[cell_nb]["spike_trains"])
    ax_rast.set(title = "Raster plot", ylabel='N Repetitions')

    ax_psth = axs[1]
    width = (raster_data[cell_nb]["repeated_sequences_times"][0][0]/600)
    ax_psth.bar(np.linspace(0,raster_data[cell_nb]["repeated_sequences_times"][0][0],int(nb_frames_by_sequence/2))+width/2, raster_data[cell_nb]["psth"], width=1.3*width)
    ax_psth.set(xlabel='Time in sec', ylabel='Firing rate (spikes/s)')

    plt.subplots_adjust(wspace=0, hspace=0)
    fig_file = os.path.normpath(os.path.join(fig_directory,f'Cluster_{cell_nb}.png'))
    plt.savefig(fig_file, dpi=fig.dpi)
    plt.clf()
    plt.close()
    
"""
    Output
    
    Save :
    
    ""{phy_directory}/clusters_rasters/Cluster_{Cluster_number}.png" for each found clusters in phy's files
"""    

print('\n\t\t\t------ End Of Cell ------')

Spike extraction: 


  0%|          | 0/25264531 [00:00<?, ?it/s]

Cluster to Spikes matching: 


  0%|          | 0/570 [00:00<?, ?it/s]

0 : 00_Checkerboard_30ND50%_20pix30checks_30Hz
1 : 01_Checkerboard_30ND50%_16pix40checks_30Hz
2 : 02_DG_30ND50%_2sT_50Hz
3 : 03_Chirp_20reps_30ND50%_50Hz
4 : 04_Flicker_BeforeDrugs_30ND50%_1Hz
5 : 05_VDH_Synchro+MultiSpots(bright)_N8_Z(-35)_30ND50%_40Hz
6 : 06_VDH_Synchro_N10_Z(-35)_30ND50%_40Hz
7 : 07_Flicker_LAP4+ACET_t+10_30ND50%_1Hz
8 : 08_HoloStim1_LAP4+ACET_N8_Z(-35)
9 : 09_HoloStim1_LAP4+ACET_N15_Z(-30)
10 : 10_OptoStim1_LAP4+ACET_15ND50%_1Hz
11 : 11_OptoStim1_LAP4+ACET_5ND50%_1Hz
12 : 12_HoloStim2_GRF_t30_N15_Z(-30)
13 : 13_OptoStim2_GRF_t35_15ND50%_1Hz
14 : 14_OptoStim2_GRF_t40_5ND50%_1Hz
15 : 15_HoloStim3_SR95531_t30_N15_Z(-30)
16 : 16_OptoStim3_SR95531_t35_15ND50%_1Hz
17 : 17_OptoStim3_SR95531_t40_5ND50%_1Hz
18 : 18_HoloStim3_18BG_t5_N15_Z(-30)
19 : 19_OptoStim3_18BG_t10_15ND50%_1Hz
20 : 20_OptoStim3_18BG_t15_5ND50%_1Hz
21 : 21_HoloStim3_18BG_t20_N15_Z(-30)

Select the checkerboard recording  for the raster plots: 1
Building Raster plots : 


  0%|          | 0/570 [00:00<?, ?it/s]

Saving rasters plots of all clusters :


  0%|          | 0/570 [00:00<?, ?it/s]


			------ End Of Cell ------


In [25]:
import params
out = f"phy template-gui {params.phy_directory}/params.py"

!env QTWEBENGINE_CHROMIUM_FLAGS="--single-process" {out}

17:54:24.170 [W] model:625            Unreferenced clusters found in spike_clusters (generally not a problem)
17:54:25.600 [W] model:667            Skipping spike waveforms that do not exist, they will be extracted on the fly from the raw data as needed.


### Cell 7 : Extract data per neurons

#### <center>REQUIRES CELL 1 RUN</center>

Extract all data from phy numpy variables. Create&save a dictionnary containg spikes times in sec for each neuron splited by recording. Depending on your experiment, this can take severeal minutes.

In [26]:
"""
    Variable
    
    You will find here all variables used in this notebook cell. They should always refere to your 'params.py' file
    except if you want to manually change some variable only for this run (i.e. debugging). You may have to add those
    variable into the function you want to adapt as only the minimal amount of var are currently given to functions as inputs.
"""

# Name of your experiment
exp = params.exp

#Path to the folder with the phy output
phy_directory = params.phy_directory

#Path to where data should be saved
output_directory = params.output_directory

#Path to raw recording files
recording_directory = params.recording_directory

#Frequency of sampling of the mea
fs = params.fs


"""
    Processing
"""

rec_onsets    = recording_onsets(recording_names, path = recording_directory)  

# Get cells index and number
cluster_number , good_clusters = extract_cluster_groups(phy_directory)
print("There are {} good clusters ({} clusters in total)\n".format(len(good_clusters), len(cluster_number)))


# Extract the spike times from the spike sorting files. This can take a few minutes.
print('Spike extraction: ')
all_spike_times = extract_all_spike_times_from_phy(phy_directory)

print('\n')
print('Spike division in recordings per neuron:')
# create a dictionary with another dictionary for each good cluster
good_data = split_spikes_by_recording(all_spike_times, good_clusters, rec_onsets)


# Save the spike data. This can take a few minutes.
good_data_file_name = os.path.join(output_directory,r'{}_fullexp_neurons_data.pkl'.format(exp))
save_obj(good_data,good_data_file_name)

"""
    Output
    
    data (dict) : key 'cluster_id' --> (dict) key 'recording_name' --> This neuron & this recording spikes times in sec

"""

print('\n\t\t\t------ End Of Cell ------')

There are 276 good clusters (570 clusters in total)

Spike extraction: 


  0%|          | 0/25264531 [00:00<?, ?it/s]



Spike division in recordings per neuron:


  0%|          | 0/276 [00:00<?, ?it/s]


			------ End Of Cell ------
